<a href="https://colab.research.google.com/github/ismiinaimah/Intelligent-Integrated-Smart-Farming-Ecosystem/blob/main/Smart_Farming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr
import pandas as pd

# Inisialisasi stok awal
stock = {
    'seed': 1000,
    'fertilizer': 500,
    'pesticide': 200
}

# Inisialisasi riwayat transaksi
transaction_history = []

# Penjual berdasarkan produk
sellers = {
    'seed': 'PT Seed Innovators Indonesia',
    'fertilizer': 'PT NutriGrow Fertilizers',
    'pesticide': 'PT NutriGrow Fertilizers'
}

# Fungsi untuk monitoring sensor
def sensor_monitoring(soil_moisture, temperature, humidity):
    if soil_moisture < 30:
        seed_recommendation = 50
        fertilizer_recommendation = 20
        pesticide_recommendation = 10
    else:
        seed_recommendation = 30
        fertilizer_recommendation = 10
        pesticide_recommendation = 5

    recommendations = {
        'Seed (kg)': seed_recommendation,
        'Fertilizer (kg)': fertilizer_recommendation,
        'Pesticide (kg)': pesticide_recommendation
    }

    return pd.DataFrame([recommendations])  # Mengembalikan DataFrame untuk tampilan tabel

# Fungsi untuk mengelola transaksi
def handle_transaction(product_type, quantity):
    if product_type not in stock:
        return "Produk tidak valid!", stock, transaction_history

    if stock[product_type] < quantity:
        return "Stok tidak mencukupi!", stock, transaction_history

    # Update stok
    stock[product_type] -= quantity

    # Log transaksi
    seller = sellers[product_type]
    transaction = {
        'Product': product_type,
        'Quantity': quantity,
        'Seller': seller
    }
    transaction_history.append(transaction)

    return f"Transaksi berhasil dengan {seller} untuk {quantity} unit {product_type}.", stock, transaction_history

# Fungsi untuk tab sensor
def sensor_tab(soil_moisture, temperature, humidity):
    return sensor_monitoring(soil_moisture, temperature, humidity)

# Fungsi untuk tab transaksi
def transaction_tab(product_type, quantity):
    transaction_status, updated_stock, _ = handle_transaction(product_type, quantity)
    updated_stock_df = pd.DataFrame(list(updated_stock.items()), columns=["Product", "Stock"])  # Konversi stok ke tabel
    return transaction_status, updated_stock_df  # Mengembalikan status transaksi dan stok dalam bentuk tabel

# Fungsi untuk tab riwayat transaksi
def history_tab():
    return pd.DataFrame(transaction_history)  # Mengembalikan riwayat transaksi dalam bentuk DataFrame

# Konfigurasi antarmuka Gradio
title = "Sistem Intelligent Integrated Smart Farming Ecosystem"

def tab1():
    with gr.Row():
        with gr.Column():
            soil_moisture = gr.Slider(0, 100, label="Soil Moisture (%)")
            temperature = gr.Slider(-10, 50, label="Temperature (°C)")
            humidity = gr.Slider(0, 100, label="Humidity (%)")
            submit_btn = gr.Button("Submit")
        with gr.Column():
            recommendations = gr.Dataframe(label="Rekomendasi untuk Benih, Pupuk, dan Pestisida", interactive=False)

        submit_btn.click(sensor_monitoring, inputs=[soil_moisture, temperature, humidity], outputs=[recommendations])

def tab2():
    with gr.Row():
        with gr.Column():
            product_type = gr.Dropdown(['seed', 'fertilizer', 'pesticide'], label="Pilih Produk")
            quantity = gr.Number(label="Jumlah")
            transaction_btn = gr.Button("Submit")
        with gr.Column():
            transaction_status = gr.Text(label="Status Transaksi")
            updated_stock = gr.Dataframe(label="Stok Tersedia", interactive=False)

        transaction_btn.click(
            transaction_tab,
            inputs=[product_type, quantity],
            outputs=[transaction_status, updated_stock]
        )

def tab3():
    with gr.Row():
        with gr.Column():
            transaction_log = gr.Dataframe(label="Riwayat Transaksi", interactive=False)
            refresh_btn = gr.Button("Refresh")

        # Tombol refresh untuk memperbarui riwayat secara manual
        refresh_btn.click(lambda: pd.DataFrame(transaction_history), inputs=[], outputs=[transaction_log])

    return transaction_log

with gr.Blocks(title=title) as interface:
    gr.Markdown(f"# {title}")
    with gr.Tab("Sensor Monitoring"):
        tab1()
    with gr.Tab("Transaction Management"):
        tab2()
    with gr.Tab("Purchase History"):
        tab3()

# Menjalankan aplikasi
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bf461e873d61481c12.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
